In [ ]:
import torch
import torch.nn as nn

In [ ]:
# building a simple model
class Model(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)
        return out

In [ ]:
## basic working of the nn

# create dataset
features = torch.rand(10, 5)

# create model
model = Model(features.shape[1])

# forward pass
# output = model.forward(features)  # not recommended!
output = model(features)    # recommended!

print(output)

tensor([[0.3667],
        [0.3916],
        [0.2531],
        [0.3422],
        [0.3179],
        [0.2996],
        [0.3075],
        [0.2995],
        [0.3557],
        [0.3395]], grad_fn=<SigmoidBackward0>)


In [ ]:
model.linear.weight

Parameter containing:
tensor([[-0.4343, -0.4038, -0.0062, -0.3823,  0.3513]], requires_grad=True)

In [ ]:
model.linear.bias

Parameter containing:
tensor([-0.3160], requires_grad=True)

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
summary(model, input_size=(10, 5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 1]                   6
├─Sigmoid: 1-2                           [10, 1]                   --
Total params: 6
Trainable params: 6
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

In [ ]:
# new model with 5 inputs and 1 hidden layer with 3 neurons and one output neuron
class Model(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        # self.linear1 = nn.Linear(num_features, 3)
        # self.relu = nn.ReLU()
        # self.linear2 = nn.Linear(3, 1)
        # self.sigmoid = nn.Sigmoid()
        self.network = nn.Sequential(
            nn.Linear(num_features, 3),
            nn.ReLU(),
            nn.Linear(3, 1),
            nn.Sigmoid()
        )

    def forward(self, features):
        # out = self.linear1(features)
        # out = self.relu(out)
        # out = self.linear2(out)
        # out = self.sigmoid(out)
        out = self.network(features)
        return out

# sequential handles the forward pass of data through each layer automatically
# hance no need to manually pass the data through each layer

In [ ]:
model = Model(features.shape[1])
output = model(features)

In [ ]:
output

tensor([[0.3639],
        [0.3659],
        [0.3332],
        [0.3751],
        [0.3871],
        [0.3656],
        [0.3726],
        [0.3467],
        [0.3593],
        [0.3878]], grad_fn=<SigmoidBackward0>)

In [ ]:
summary(model, input_size=(10, 5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Sequential: 1-1                        [10, 1]                   --
│    └─Linear: 2-1                       [10, 3]                   18
│    └─ReLU: 2-2                         [10, 3]                   --
│    └─Linear: 2-3                       [10, 1]                   4
│    └─Sigmoid: 2-4                      [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

# New model for the same data previously used (training_pipeline.py)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")
# df.head()

In [ ]:
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)
# df.head()

In [ ]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

## New Model

In [ ]:
# the new model using torch nn
class myNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)
        return out

    # def loss(self, y_pred, y_true):
    #     epsilon = 1e-9
    #     y_pred = torch.clamp(y_pred, min=epsilon, max=1-epsilon)
    #     loss = -(y_true * torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred)).mean()
    #     return loss

    # def update_params(self, learning_rate=0.001):
    #     with torch.no_grad():
    #         self.linear.weight -= learning_rate * self.linear.weight.grad
    #         self.linear.bias -= learning_rate * self.linear.bias.grad
    #     self.linear.weight.grad.zero_()
    #     self.linear.bias.grad.zero_()

## New Training Pipeline

In [ ]:
# important params
learning_rate = 0.1
epochs = 101

# loss function
loss_function = nn.BCELoss()

# create model
model = myNN(X_train_tensor.shape[1])

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# train the model
for epoch in range(epochs):
    # forward pass
    y_pred = model(X_train_tensor)
    # loss calculation
    loss = loss_function(y_pred, y_train_tensor.view(-1, 1))
    # backward pass
    loss.backward()
    # update params
    optimizer.step()
    optimizer.zero_grad()

    if epoch % 10 == 0:
        print(f"epoch: {epoch}, loss: {loss}")

epoch: 0, loss: 0.6427109241485596
epoch: 10, loss: 0.2420252561569214
epoch: 20, loss: 0.18601807951927185
epoch: 30, loss: 0.15936191380023956
epoch: 40, loss: 0.14321176707744598
epoch: 50, loss: 0.13218918442726135
epoch: 60, loss: 0.12408481538295746
epoch: 70, loss: 0.11781545728445053
epoch: 80, loss: 0.11278443038463593
epoch: 90, loss: 0.10863412171602249
epoch: 100, loss: 0.10513587296009064


In [ ]:
y_pred[:5]

tensor([[0.1380],
        [1.0000],
        [0.0017],
        [0.0786],
        [0.0048]], grad_fn=<SliceBackward0>)

In [ ]:
model.linear.weight, model.linear.bias

(Parameter containing:
 tensor([[ 0.4084,  0.2255,  0.2252,  0.4479,  0.0178,  0.1531,  0.2742,  0.3291,
           0.1489, -0.2157,  0.3449, -0.0527,  0.3683,  0.3158, -0.0422, -0.2103,
          -0.0928,  0.2112, -0.1398, -0.1443,  0.4101,  0.5818,  0.4757,  0.2383,
           0.4105,  0.2303,  0.2861,  0.5069,  0.2804,  0.0520]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.4260], requires_grad=True))

### Evaluating the Model

In [ ]:
y_pred = model.forward(X_test_tensor)
loss = loss_function(y_pred, y_test_tensor.view(-1, 1))
acc = (y_pred.round() == y_test_tensor).float().mean()
print(f"loss = {loss}, accuracy = {acc}")

loss = 0.08709141612052917, accuracy = 0.5323176383972168


# batch-wise training

In [ ]:
learning_rate = 0.1
epochs = 20

loss_function = nn.BCELoss()

model = myNN(X_train_tensor.shape[1])

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

batch_size = 32
n_samples = len(X_train_tensor)

for epoch in range(epochs):
    for start_idx in range(0, n_samples, batch_size):
        end_idx = min(start_idx + batch_size, n_samples)
        X_batch = X_train_tensor[start_idx:end_idx]
        y_batch = y_train_tensor[start_idx:end_idx]

        y_pred = model(X_batch)
        loss = loss_function(y_pred, y_batch.view(-1, 1))

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"epoch: {epoch+1}, loss: {loss.item()}")

epoch: 1, loss: 0.21400918066501617
epoch: 2, loss: 0.14821168780326843
epoch: 3, loss: 0.11877468973398209
epoch: 4, loss: 0.101157546043396
epoch: 5, loss: 0.08913639932870865
epoch: 6, loss: 0.08027879893779755
epoch: 7, loss: 0.07341314107179642
epoch: 8, loss: 0.067896768450737
epoch: 9, loss: 0.06334400922060013
epoch: 10, loss: 0.05950772017240524
epoch: 11, loss: 0.05622119829058647
epoch: 12, loss: 0.05336746200919151
epoch: 13, loss: 0.050861649215221405
epoch: 14, loss: 0.04864048212766647
epoch: 15, loss: 0.04665568843483925
epoch: 16, loss: 0.04486974701285362
epoch: 17, loss: 0.04325288534164429
epoch: 18, loss: 0.04178124666213989
epoch: 19, loss: 0.04043537378311157
epoch: 20, loss: 0.03919930011034012


In [ ]:
model.linear.weight, model.linear.bias

(Parameter containing:
 tensor([[ 0.4017,  0.5319,  0.3615,  0.4417,  0.3128,  0.0488,  0.4735,  0.4814,
           0.1397, -0.1881,  0.5277, -0.0532,  0.2967,  0.5415,  0.0383, -0.1611,
          -0.0652, -0.0908, -0.0713, -0.4082,  0.7977,  0.7638,  0.5474,  0.7086,
           0.3761,  0.3105,  0.4907,  0.6386,  0.3114, -0.0408]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.4919], requires_grad=True))

### Evaluating the model with batch-wise training

In [ ]:
y_pred = model(X_test_tensor)
loss = loss_function(y_pred, y_test_tensor.view(-1, 1))
acc = (y_pred.round() == y_test_tensor.view(-1, 1)).float().mean()
print(f"loss: {loss.item()}, acc: {acc.item()}")

loss: 0.06638342887163162, acc: 0.9824561476707458
